### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH' # your Foursquare ID
CLIENT_SECRET = '0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH
CLIENT_SECRET: 0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER


### Let's assume that you are staying at the Hilton hotel. So let's start by converting the Hilton Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = 'Needham\'s Point, St. Michael, Bridgetown, Barbados'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

13.0783977 -59.6114459


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 5000 metres from the Hilton Hotel.

In [4]:
search_query = 'Italian'
radius = 5000
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.0783977,-59.6114459&v=20180604&query=Italian&radius=5000&limit=30'
url

'https://api.foursquare.com/v2/venues/search?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.0783977,-59.6114459&v=20180604&query=Italian&radius=5000&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce95a47db04f52f65f645ce'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '4c8c14d68018a1cd8979e9d2',
    'location': {'address': 'Hastings Main Road',
     'cc': 'BB',
     'city': 'Barbados',
     'country': 'Barbados',
     'distance': 995,
     'formattedAddress': ['Hastings Main Road', 'Barbados', 'Barbados'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 13.07669389251161,
       'lng': -59.60243639378871}],
     'lat': 13.07669389251161,
     'lng': -59.60243639378871},
    'name': 'Mama Mia Italian Deli & Pizzeria',
    'referralId': 'v-1558796871',
    'venuePage': {'id': '70591572'}},
   {'categories': [{'icon': {'prefix': 'https

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,name,referralId,venuePage.id
0,"[{'pluralName': 'Italian Restaurants', 'icon':...",False,4c8c14d68018a1cd8979e9d2,Hastings Main Road,BB,Barbados,Barbados,995,"[Hastings Main Road, Barbados, Barbados]","[{'lat': 13.07669389251161, 'label': 'display'...",13.076694,-59.602436,Mama Mia Italian Deli & Pizzeria,v-1558796871,70591572
1,"[{'pluralName': 'Italian Restaurants', 'icon':...",False,55d55d28498edf2c6a89ed80,Hastings,BB,Christ Church,Barbados,1141,"[Hastings, Christ Church, Barbados]","[{'lat': 13.075981523385389, 'label': 'display...",13.075982,-59.601219,Buzo Osteria Italiana,v-1558796871,NaN


#### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,id
0,Mama Mia Italian Deli & Pizzeria,Italian Restaurant,Hastings Main Road,BB,Barbados,Barbados,995,"[Hastings Main Road, Barbados, Barbados]","[{'lat': 13.07669389251161, 'label': 'display'...",13.076694,-59.602436,4c8c14d68018a1cd8979e9d2
1,Buzo Osteria Italiana,Italian Restaurant,Hastings,BB,Christ Church,Barbados,1141,"[Hastings, Christ Church, Barbados]","[{'lat': 13.075981523385389, 'label': 'display...",13.075982,-59.601219,55d55d28498edf2c6a89ed80


#### Let's visualize the Italian restaurants that are nearby.

In [9]:
dataframe_filtered.name

0    Mama Mia Italian Deli & Pizzeria
1               Buzo Osteria Italiana
Name: name, dtype: object

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Hilton Hotel

# add a red circle marker to represent the Hilton Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hilton Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Mama Mia_

In [11]:
venue_id = '4c8c14d68018a1cd8979e9d2' # ID of Mama Mia Italian Deli & Pizzeria
url = 'https://api.foursquare.com/v2/venues/4c8c14d68018a1cd8979e9d2?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&v=20180604'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c8c14d68018a1cd8979e9d2?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&v=20180604'

#### Send GET request for result

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['createdAt', 'listed', 'likes', 'price', 'popular', 'canonicalUrl', 'photos', 'hereNow', 'beenHere', 'verified', 'id', 'contact', 'ok', 'rating', 'colors', 'allowMenuUrlEdit', 'bestPhoto', 'location', 'specials', 'ratingSignals', 'stats', 'categories', 'pageUpdates', 'name', 'reasons', 'attributes', 'dislike', 'venuePage', 'tips', 'page', 'timeZone', 'inbox', 'ratingColor', 'shortUrl'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 5,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'Yes'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'}],
    'name': 'Menus',
    'summary': 'Dinner, Lunch & more',
    'type': 'serves'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1346609056,
  'height'

### B. Get the venue's overall rating

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.4


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [14]:
venue_id = '55d55d28498edf2c6a89ed80' # ID of Buzo's
url = 'https://api.foursquare.com/v2/venues/55d55d28498edf2c6a89ed80?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&v=20180604'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.8


Since this restaurant has a much better rating, let's explore it further.

### C. Get the number of tips

In [15]:
result['response']['venue']['tips']['count']

15

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [16]:
## Buzo Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/55d55d28498edf2c6a89ed80/tips?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&v=20180604&limit=15'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce95a489fb6b7757e8a7f67'},
 'response': {'tips': {'count': 15,
   'items': [{'agreeCount': 2,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5a0500b56661165f57c8d039',
     'createdAt': 1510277301,
     'disagreeCount': 0,
     'id': '5a0500b56661165f57c8d039',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': "Five of us ate here and shared each other's dishes - we all agree - Buzo Osteria knows what great Italian food is all about! 10/10 would eat here again!",
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'S',
      'gender': 'none',
      'id': '112258644',
      'lastName': 'M',
      'photo': {'default': True,
       'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/blank_boy.png'}}}]}}}

#### Get tips and list of associated features.

In [17]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['todo', 'createdAt', 'user', 'likes', 'id', 'logView', 'canonicalUrl', 'authorInteractionType', 'disagreeCount', 'text', 'agreeCount', 'type', 'lang'])

#### Format column width and display all tips

In [18]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Five of us ate here and shared each other's dishes - we all agree - Buzo Osteria knows what great Italian food is all about! 10/10 would eat here again!,2,0,5a0500b56661165f57c8d039,S,M,none,112258644


## 3. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

### So, you just finished your gourmet dish at Buzo's, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Buzo's.

In [19]:
latitude = 13.0760082
longitude = -59.6032783

### Define URL

In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.0760082,-59.6032783&v=20180604&radius=500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.0760082,-59.6032783&v=20180604&radius=500&limit=30'

### Send GET request and examine results

In [21]:
import requests

In [22]:
results = requests.get(url).json()
'There are {} venues around Buzo Osteria Italiana.'.format(len(results['response']['groups'][0]['items']))

'There are 11 venues around Buzo Osteria Italiana.'

### Get relevant part of JSON

In [23]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-55d55d28498edf2c6a89ed80-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d110941735',
    'name': 'Italian Restaurant',
    'pluralName': 'Italian Restaurants',
    'primary': True,
    'shortName': 'Italian'}],
  'id': '55d55d28498edf2c6a89ed80',
  'location': {'address': 'Hastings',
   'cc': 'BB',
   'city': 'Christ Church',
   'country': 'Barbados',
   'distance': 223,
   'formattedAddress': ['Hastings', 'Christ Church', 'Barbados'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 13.075981523385389,
     'lng': -59.60121935195081}],
   'lat': 13.075981523385389,
   'lng': -59.60121935195081},
  'name': 'Buzo Osteria Italiana',
  'photos': {'count': 0, 'groups': []}}}

### Process JSON and convert it to a clean dataframe

In [24]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(11)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Buzo Osteria Italiana,Italian Restaurant,Hastings,BB,Christ Church,Barbados,NaN,223,"[Hastings, Christ Church, Barbados]","[{'lat': 13.075981523385389, 'label': 'display', 'lng': -59.60121935195081}]",13.075982,-59.601219,NaN,NaN,NaN,55d55d28498edf2c6a89ed80
1,thecoffeebean,Coffee Shop,"The Walk,",BB,NaN,Barbados,NaN,189,"[The Walk,, Barbados]","[{'lat': 13.077469350309816, 'label': 'display', 'lng': -59.60416861948301}]",13.077469,-59.604169,Welches,NaN,NaN,4c49c9633013a593c91938e3
2,Sugar Bay Barbados,Resort,Garrison Historic Area,BB,NaN,Barbados,Hastings,166,"[Garrison Historic Area (Hastings), BB15156, Barbados]","[{'lat': 13.076457476632502, 'label': 'display', 'lng': -59.60474517419773}]",13.076457,-59.604745,NaN,BB15156,Christ Church,564bf89a498e50af0c24a5d0
3,Courtyard Marriot,Hotel,"The Garrison Historic Area, Hastings. Christ Church",BB,Bridgetown,Barbados,Christ Church,217,"[The Garrison Historic Area, Hastings. Christ Church (Christ Church), Bridgetown BB15156, Barbados]","[{'lat': 13.076614823939176, 'label': 'display', 'lng': -59.60137188434601}]",13.076615,-59.601372,Bridgetown,BB15156,Saint Michael,4d3bab3e14aa8cfa12fba15e
4,Soco Hotel,Hotel,NaN,BB,NaN,Barbados,NaN,432,[Barbados],"[{'lat': 13.07451434171696, 'label': 'display', 'lng': -59.59959695815124}]",13.074514,-59.599597,NaN,NaN,NaN,51b9077b498efc48f78e3d09
5,The Garrison Savannah,Racetrack,Highway 7,BB,St. Michael,Barbados,NaN,449,"[Highway 7, St. Michael, Barbados]","[{'lat': 13.079561881202464, 'label': 'display', 'lng': -59.60525035858154}]",13.079562,-59.605250,NaN,NaN,NaN,4f191ae6a17c5eb80f525c32
6,Blue Room Sports Bar & Grill,Bar,Garrison Historic Area,BB,Hastings,Barbados,NaN,146,"[Garrison Historic Area, Hastings BB15156, Barbados]","[{'lat': 13.07639904856015, 'label': 'display', 'lng': -59.601983909065495}]",13.076399,-59.601984,NaN,BB15156,Christ Church,4e910584dab46521c151eeb6
7,Mama Mia Italian Deli & Pizzeria,Italian Restaurant,Hastings Main Road,BB,Barbados,Barbados,NaN,118,"[Hastings Main Road, Barbados, Barbados]","[{'lat': 13.07669389251161, 'label': 'display', 'lng': -59.60243639378871}]",13.076694,-59.602436,NaN,NaN,NaN,4c8c14d68018a1cd8979e9d2
8,The Savannah Hotel,Resort,Hastings Main Rd,BB,NaN,Barbados,NaN,293,"[Hastings Main Rd, Barbados]","[{'lat': 13.077437488356567, 'label': 'display', 'lng': -59.60555255170073}]",13.077437,-59.605553,NaN,NaN,Christ Church,4ed52bbc6da10f2075787508
9,Fred's,BBQ Joint,Tweedside Road,BB,Bridgetown,Barbados,NaN,224,"[Tweedside Road, Bridgetown, Barbados]","[{'lat': 13.076022232384787, 'label': 'display', 'lng': -59.60120350217638}]",13.076022,-59.601204,NaN,NaN,Saint Michael,4ec459dd8b812a20648ec047


### Let's visualize these items on the map around our location.

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Buzo's


# add Buzo's as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Buzo Osteria Italiana',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### Let's examine a similar venue in another location. Assume that you are staying at the Crane hotel. So let's start by converting the Crane Hotel's address to its latitude and longitude coordinates.

In [26]:
address = 'The Crane, St. Philip, Barbados'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

13.1059821 -59.4484396


### Let's define a query to search for Italian food that is within 5000 metres from the Crane Hotel.

In [27]:
search_query = 'Italian'
radius = 5000
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [28]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.1059821,-59.4484396&v=20180604&query=Italian&radius=5000&limit=30'
url

'https://api.foursquare.com/v2/venues/search?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.1059821,-59.4484396&v=20180604&query=Italian&radius=5000&limit=30'

#### Send the GET Request and examine the results

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce95a4a9fb6b775801ac007'},
 'response': {'venues': []}}

### There are no Italian restaurants within 5000 metres of the Crane Hotel.

#### Let us visualize the location of the Crane Hotel.

In [30]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Crane Hotel

# add a red circle marker to represent the Crane Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Crane Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# display map
venues_map

### Let us explore the area around the Crane Hotel. Let's start by getting the latitute and logitude of the location.

In [31]:
latitude = 13.105658
longitude = -59.4472754

### Define URL

In [32]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.105658,-59.4472754&v=20180604&radius=500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WSFCHVIQBBLNHRCVKR0OIVPJJPRAXRJGWTPXPL32JTY4DSPH&client_secret=0XAJSLYFAXJNNL4KHXT20UAIANVPOUGS1KMNSQ44NQBTY2ER&ll=13.105658,-59.4472754&v=20180604&radius=500&limit=30'

### Send GET request and examine results

In [33]:
import requests

In [34]:
results = requests.get(url).json()
'There are {} venues around the Crane Hotel.'.format(len(results['response']['groups'][0]['items']))

'There are 13 venues around the Crane Hotel.'

### Get relevent part of JSON

In [35]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4bcb96beaeaaeee1e9e93d6d-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/resort_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d12f951735',
    'name': 'Resort',
    'pluralName': 'Resorts',
    'primary': True,
    'shortName': 'Resort'}],
  'id': '4bcb96beaeaaeee1e9e93d6d',
  'location': {'address': 'The Crane',
   'cc': 'BB',
   'city': 'St. Philip',
   'country': 'Barbados',
   'distance': 184,
   'formattedAddress': ['The Crane', 'St. Philip BB18079', 'Barbados'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 13.105538443691795,
     'lng': -59.44557617504648}],
   'lat': 13.105538443691795,
   'lng': -59.44557617504648,
   'postalCode': 'BB18079',
   'state': 'Barbados'},
  'name': 'The Crane Residential Resort',
  'photos': {'count': 0, 'groups': []},
  'v

### Process JSON and convert it to a clean dataframe

In [36]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(13)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Crane Residential Resort,Resort,The Crane,BB,St. Philip,Barbados,NaN,184,"[The Crane, St. Philip BB18079, Barbados]","[{'lat': 13.105538443691795, 'label': 'display', 'lng': -59.44557617504648}]",13.105538,-59.445576,BB18079,Barbados,4bcb96beaeaaeee1e9e93d6d
1,Crane Beach,Beach,Crane Beach,BB,St. Philip,Barbados,NaN,304,"[Crane Beach, St. Philip, Barbados]","[{'lat': 13.10637435055967, 'label': 'display', 'lng': -59.44456453746355}]",13.106374,-59.444565,NaN,Barbados,4df74bd98130bd1c527e82e2
2,Zen,Japanese Restaurant,The Crane,BB,St. Philip,Barbados,NaN,298,"[The Crane, St. Philip, Barbados]","[{'lat': 13.106180815985848, 'label': 'display', 'lng': -59.44457565444356}]",13.106181,-59.444576,NaN,Barbados,4cc4ca24c844721ef87eeb01
3,Cutters,Sandwich Place,N Hwy,BB,Crane Beach,Barbados,Crane Beach,346,"[N Hwy (Crane Beach), Crane Beach, Barbados]","[{'lat': 13.108770777540197, 'label': 'display', 'lng': -59.447106732501204}]",13.108771,-59.447107,NaN,Saint Phillip,4c55b7c01b46c9b64cd6b7d0
4,Bar 1887,Bar,The Crane Resort,BB,Rices,Barbados,NaN,197,"[The Crane Resort, Rices, Barbados]","[{'lat': 13.105550112055722, 'label': 'display', 'lng': -59.44545280903118}]",13.105550,-59.445453,NaN,Saint Philip,4e4af11562e1ce23e56fcfe1
5,The Carriage House Bar,Cocktail Bar,The Crane,BB,St. Philip,Barbados,NaN,255,"[The Crane, St. Philip, Barbados]","[{'lat': 13.104259494549806, 'label': 'display', 'lng': -59.44541023887735}]",13.104259,-59.445410,NaN,Barbados,4e1a0e81a8097d08b246b5d7
6,The Village Café,Café,The Cranes,BB,NaN,Barbados,NaN,222,"[The Cranes, Barbados]","[{'lat': 13.105611469064169, 'label': 'display', 'lng': -59.44522780961435}]",13.105611,-59.445228,NaN,NaN,50202c4ae4b004a3679ff414
7,L'Azure,Seafood Restaurant,The Crane Residential Resort,BB,St. Philip,Barbados,NaN,289,"[The Crane Residential Resort, St. Philip, Barbados]","[{'lat': 13.105892281656464, 'label': 'display', 'lng': -59.44461440528041}]",13.105892,-59.444614,NaN,Barbados,4e77eca8b61ce1c89847de32
8,D'Onofrio's,Italian Restaurant,The Crane Residential Resort,BB,St. Philip,Barbados,NaN,233,"[The Crane Residential Resort, St. Philip, Barbados]","[{'lat': 13.105569104536473, 'label': 'display', 'lng': -59.44512172891144}]",13.105569,-59.445122,NaN,Barbados,4de82d1c88774aa92c994217
9,"Foul Bay, St. Philip",Beach,NaN,BB,NaN,Barbados,NaN,153,[Barbados],"[{'lat': 13.104449336776973, 'label': 'display', 'lng': -59.447965249085065}]",13.104449,-59.447965,NaN,NaN,4cba1ae243ec6dcb811ca231


### Let's visualize these items on the map around our location.

In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Crane Hotel


# add Buzo's as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='The Crane Hotel',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### As can be seen, most of the venues of interest in the area are located in the Crane Hotel property itself.